In [1]:
import os, glob
import torchio as tio
import matplotlib.pyplot as plt

/home/trojan/anaconda3/envs/art_seg/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_dir = os.path.abspath("./data/train")
volumes_dir = os.path.join(data_dir + "/volumes")
masks_dir = os.path.join(data_dir + "/masks")
volumes_paths = sorted(glob.glob(volumes_dir + "/" + "*.nii.gz"))
masks_paths = sorted(glob.glob(masks_dir + "/" + "*.nii.gz"))

In [3]:
####### Testing on just one image ###########

In [4]:
subjects = []
for volume_path in volumes_paths:
    subject = tio.ScalarImage(volume_path)
    subjects.append(subject)

In [5]:
print(len(subjects))

17


In [6]:
sample = subjects[0]

In [17]:
"""
Testing various transformations and determining correct argument ranges
"""

# transform = tio.RandomFlip(axes=(0,1), flip_probability=1)
# transform = tio.RandomAnisotropy(axes=(0,1,2), downsampling=6)
# transform = tio.RandomMotion()
# transform = tio.RandomGhosting()
# transform = tio.RandomSpike()
# transform = tio.RandomBiasField()
# transform = tio.RandomBlur(std=(0,3))
# transform = tio.RandomNoise(std=(0,400))  # (std=256,256)
# transform = tio.RandomSwap(patch_size=30)
# transform = tio.RandomGamma(log_gamma=(-0.5, 0.5))
# transform = tio.RandomAffine(scales=(0.9, 1.2), degrees=15)
# transform = tio.Resample(3)


# Check canonical etc.

In [18]:
output = transform(sample)

In [19]:
output.save("./aug_data/test.nii.gz")

In [24]:
# To apply one by one in loop
transforms_dict = {
    "random-flip": tio.RandomFlip(axes=(0,1), flip_probability=1),
    "random-anisotropy": tio.RandomAnisotropy(axes=(0,1,2), downsampling=6),
    "random-motion": tio.RandomMotion(),
    "random-ghosting": tio.RandomGhosting(),
    "random-spike": tio.RandomSpike(),
    "random-biasfield": tio.RandomBiasField(),
    "random-blur": tio.RandomBlur(std=(0,3)),
    "random-noise": tio.RandomNoise(std=(0,400)),
    "random-swap": tio.RandomSwap(patch_size=30),
    "random-gamma": tio.RandomGamma(log_gamma=(-0.5, 0.5)),
    "random-affine": tio.RandomAffine(scales=(0.9, 1.2), degrees=15),
    "resample": tio.Resample(3)
}

# print(transforms_dict.values())
for name,transform in transforms_dict.items():
    output = transform(sample)
    output.save(f"./aug_data/test-{name}.nii.gz")

In [ ]:
###########################################

In [ ]:
########### Testing on image and mask ##################

In [189]:
subjects = []
for (volume_path, mask_path) in zip(volumes_paths, masks_paths):
    subject = tio.Subject(
        vol=tio.ScalarImage(volume_path),
        mask=tio.LabelMap(mask_path),
    )
    subjects.append(subject)
dataset = tio.SubjectsDataset(subjects)
print('Dataset size:', len(dataset), 'subjects')

Dataset size: 17 subjects


In [190]:
transform = tio.RandomAffine(scales=(0.9, 1.2), degrees=15)

In [24]:
transform = tio.Compose([
    tio.RandomFlip(axes=(0,1), flip_probability=1),
    tio.RandomMotion(p=0.5),
    tio.RandomBiasField(p=0.3),
    tio.RandomNoise(p=0.5),
    tio.RandomFlip(),
    tio.OneOf({
        tio.RandomAffine(
            scales=(0.9, 1.2),
            degrees=15
        ): 0.8,
        tio.RandomElasticDeformation(): 0.2,
    }),
])

In [191]:
sample = subjects[0]

In [192]:
print(sample)

Subject(Keys: ('vol', 'mask'); images: 2)


In [193]:
output = transform(sample)

In [194]:
temp = list(output.values())

In [195]:
print(temp)

[ScalarImage(shape: (1, 512, 512, 510); spacing: (0.40, 0.40, 0.50); orientation: LPS+; dtype: torch.FloatTensor; memory: 510.0 MiB), LabelMap(shape: (1, 512, 512, 510); spacing: (0.40, 0.40, 0.50); orientation: LPS+; dtype: torch.FloatTensor; memory: 510.0 MiB)]


In [196]:
print(temp[0])

ScalarImage(shape: (1, 512, 512, 510); spacing: (0.40, 0.40, 0.50); orientation: LPS+; dtype: torch.FloatTensor; memory: 510.0 MiB)


In [197]:
print(temp[1])

LabelMap(shape: (1, 512, 512, 510); spacing: (0.40, 0.40, 0.50); orientation: LPS+; dtype: torch.FloatTensor; memory: 510.0 MiB)


In [198]:
temp[0].save("./aug_data/test.nii.gz")
temp[1].save("./aug_data/test_mask.nii.gz")

In [ ]:
################################################

In [ ]:
############# Apply different transformations in loop ########################

In [199]:
subjects = []
for (volume_path, mask_path) in zip(volumes_paths, masks_paths):
    subject = tio.Subject(
        vol=tio.ScalarImage(volume_path),
        mask=tio.LabelMap(mask_path),
    )
    subjects.append(subject)
dataset = tio.SubjectsDataset(subjects)
print('Dataset size:', len(dataset), 'subjects')

Dataset size: 17 subjects


In [200]:
sample = subjects[0]

In [203]:
# Need to assign probabilities here for tio.OneOf 
transform_dict = {
    tio.Resample(3),
    tio.RandomMotion(),
    tio.RandomBiasField(),
    tio.RandomNoise(std=(0,400)),
    tio.RandomFlip(axes=(0,1), flip_probability=1),
    tio.RandomAnisotropy(axes=(0,1,2), downsampling=6),
    tio.RandomGhosting(),
    tio.RandomSpike(),
    tio.RandomBlur(std=(0,3)),
    tio.RandomSwap(patch_size=30),
    tio.RandomGamma(log_gamma=(-0.5, 0.5)),
    tio.OneOf({
        tio.RandomAffine(
            scales=(0.9, 1.2),
            degrees=(15, 15)
        ): 0.8,
        tio.RandomElasticDeformation(): 0.2,
    }),
} 

In [204]:
transform_compose = tio.Compose([
    tio.Resample(p=0.3, target=3),
    tio.RandomMotion(p=0.2),
    tio.RandomBiasField(p=0.3),
    tio.RandomNoise(p=0.5, std=(0,400)),
    tio.RandomFlip(axes=(0,1), flip_probability=0.5),
    tio.RandomAnisotropy(p=0.3, axes=(0,1,2), downsampling=6),
    tio.RandomGhosting(p=0.2),
    tio.RandomSpike(p=0.3),
    tio.RandomBlur(p=0.3, std=(0,3)),
    tio.RandomSwap(p=0.3, patch_size=30),
    tio.RandomGamma(p=0.3, log_gamma=(-0.5, 0.5)),
    tio.OneOf(p=0.3, transforms = {
        tio.RandomAffine(
            scales=(0.9, 1.2),
            degrees=(15, 15)
        ): 0.8,
        tio.RandomElasticDeformation(): 0.2,
    }),
])

In [205]:
transform = tio.OneOf({transform_compose: 0.2, tio.OneOf(transform_dict): 0.8})

In [206]:
for i in range (0, 10):
    output = transform(sample)
    temp = list(output.values())
    temp[0].save(f"./aug_data/test_{i}.nii.gz")
    temp[1].save(f"./aug_data/test_mask_{i}.nii.gz")